In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py

In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json

In [ ]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/parse-faq.ipynb

In [43]:
import minsearch
import json

In [44]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [45]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [46]:
index = minsearch.Index(
    text_fields=['text', 'section', 'question'],
    keyword_fields=['course']
)

In [47]:
q = 'The course has already started, can I still enroll?'

In [48]:
index.fit(documents)

In [49]:
boost = {'question': 3.0, 'section': 0.5}
filter = {'course':'data-engineering-zoomcamp'}
results = index.search(
    query=q,
    filter_dict=filter,
    boost_dict=boost,
    num_results=5
)

In [50]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [51]:
from google import genai
from google.genai import types

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

contents = types.Content(
  role='user',
  parts=[types.Part.from_text(text='Is it too late to join the course?')]
)

response = client.models.generate_content(
    model="gemini-2.5-flash", contents=contents
)

client.chats.create
print(response.text)

It's impossible for me to say without more information!

To find out if it's too late to join the course, please tell me:

1.  **What course are you interested in?** (e.g., "Introduction to Python," "Digital Marketing Fundamentals")
2.  **Where is it being offered?** (e.g., "XYZ University," "Coursera," "Local Community College," "A specific online platform")
3.  **Do you know when it was supposed to start, or when you heard about it?**

Once I have that information, I can tell you the best way to check, or if I can find the details myself.

**In general, here's how you can usually find out:**

*   **Check the official course website or registration page:** Look for sections like "Enrollment," "Admissions," "Deadlines," "How to Register," or "FAQs."
*   **Look for a contact email or phone number:** Reach out directly to the admissions, registration, or course coordinator.
*   **Check for a waitlist option:** Even if registration is closed, some courses offer a waitlist.
*   **See if th

In [52]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}"

prompt = prompt_template.format(question=q, context=context)
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: The course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final

In [53]:
contents = types.Content(
  role='user',
  parts=[types.Part.from_text(text=prompt)]
)

response = client.models.generate_content(
    model="gemini-2.5-flash", contents=contents
)

client.chats.create
print(response.text)

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects.


In [72]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    filter = {'course':'data-engineering-zoomcamp'}

    results = index.search(
        query=query,
        filter_dict=filter,
        boost_dict=boost,
        num_results=5
    )

    return results

In [89]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}"

    prompt = prompt_template.format(question=query, context=context)
    return prompt

In [90]:
def llm(prompt):
    contents = types.Content(
      role='user',
      parts=[types.Part.from_text(text=prompt)]
    )
    
    response = client.models.generate_content(
        model="gemini-2.5-flash", contents=contents
    )
    
    client.chats.create
    return response.text

In [92]:
query = 'BigQuery returns an error when i try to run dbt run'
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)
print(answer)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT or what you can infer from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: BigQuery returns an error when i try to run dbt run

CONTEXT:
section: Module 4: analytics engineering with dbt
question: BigQuery returns an error when i try to run ‘dbt run’:
answer: My taxi data was loaded into gcs with etl_web_to_gcs.py script that converts csv data into parquet. Then I placed raw data trips into external tables and when I executed dbt run I got an error message: Parquet column 'passenger_count' has type INT64 which does not match the target cpp_type DOUBLE. It is because several columns in files have different formats of data.
When I added df[col] = df[col].astype('Int64') transformation to the columns: passenger_count, payment_type, RatecodeID, VendorID, trip_type it went ok. Several people also fa

In [93]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer